In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("input/Normalized.csv",nrows=20000)
#df = df.drop(["text"],axis=1)

df.head()

,text,category,tokens
0,<p>I am analyzing data (size/survival) of two ...,post,"['i', 'am', 'analyzing', 'data', 'size', 'surv..."
1,@Tim Does it mean that I am interpreting the N...,comment,"['tim', 'does', 'it', 'mean', 'that', 'i', 'am..."
2,"Well, looking back on that, it is only true th...",comment,"['well', 'looking', 'back', 'on', 'that', 'it'..."
3,Have you considered using the post-interventio...,comment,"['have', 'you', 'considered', 'using', 'the', ..."
4,"<p>Perhaps you're already aware of this, but C...",post,"['perhaps', 'you', 'are', 'already', 'aware', ..."


In [3]:
p =  df['tokens'].apply(lambda x: x.replace('[','').replace(']','')) 

In [4]:
all_text = p.to_list()

In [5]:
import re
all_text = re.sub("[^a-zA-Z]", " ", str(all_text)) 

In [6]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        seq = seq.replace("'","")
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences = create_seq(str(all_text))

Total Sequences: 11216571


In [7]:
sequences

['   i    am    analyzing    data',
 '  i    am    analyzing    data ',
 ' i    am    analyzing    data  ',
 'i    am    analyzing    data   ',
 '    am    analyzing    data    ',
 '   am    analyzing    data    s',
 '  am    analyzing    data    si',
 ' am    analyzing    data    siz',
 'am    analyzing    data    size',
 'm    analyzing    data    size ',
 '    analyzing    data    size  ',
 '   analyzing    data    size   ',
 '  analyzing    data    size    ',
 ' analyzing    data    size    s',
 'analyzing    data    size    su',
 'nalyzing    data    size    sur',
 'alyzing    data    size    surv',
 'lyzing    data    size    survi',
 'yzing    data    size    surviv',
 'zing    data    size    surviva',
 'ing    data    size    survival',
 'ng    data    size    survival ',
 'g    data    size    survival  ',
 '    data    size    survival   ',
 '   data    size    survival    ',
 '  data    size    survival    o',
 ' data    size    survival    of',
 'data    size    survival  

In [ ]:
#t = sequences.copy()

In [ ]:
#u = [elem.replace("[\"",'').replace("]\"", '') for elem in t]

In [ ]:
#v = [elem.replace("\"",'').replace("\"", '') for elem in p]

In [ ]:
#p

#Encoding sequences

In [8]:
# create a character mapping index
chars = sorted(list(set(all_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [9]:
#Train test split
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Using TensorFlow backend.


Train shape: (10094913, 30) Val shape: (1121658, 30)


In [10]:
#Model Building
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=10, verbose=2, validation_data=(X_val, y_val))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            1550      
_________________________________________________________________
gru_1 (GRU)                  (None, 150)               90450     
_________________________________________________________________
dense_1 (Dense)              (None, 31)                4681      
Total params: 96,681
Trainable params: 96,681
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 10094913 samples, validate on 1121658 samples
Epoch 1/10
 - 16850s - loss: 0.9453 - acc: 0.7184 - val_loss: 0.8764 - val_acc: 0.7369
Epoch 2/10
 - 9096s - loss: 1.5568 - acc: 0.5878 - val_loss: 1.8222 - val_acc: 0.5104
Epoch 3/10
 - 51045s - loss: 2.0619 - acc: 0.4912 - val_loss: 1.8956 - val_acc: 0.4904
Epoch 4/10
 - 9417s - loss: 2.1240 - acc: 0.4864 - val_loss: 1.9778 - val_acc: 0.4815
Epoch 5/10
 - 11269s - loss: 2.1267 - acc: 0.4868 - val_loss: 1.9524 - val_acc: 0.4879
Epoch 6/10
 - 8867s - loss: 2.1363 - acc: 0.4838 - val_loss: 1.9505 - val_acc: 0.4756
Epoch 7/10
 - 50120s - loss: 2.1150 - acc: 0.4844 - val_loss: 1.9209 - val_acc: 0.4860
Epoch 8/10
 - 13594s - loss: 2.1361 - acc: 0.4835 - val_loss: 1.9745 - val_acc: 0.4835
Epoch 9/10
 - 26623s - loss: 2.1262 - acc: 0.4838 - val_loss: 1.9647 - val_acc: 0.4931
Epoch 10/10
 - 52970s - loss: 2.1158 - acc: 0.4827 - val_loss: 1.9648 - val_acc: 0.4834


In [11]:
# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text